In [39]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager


In [40]:
FBREF_HOMEPAGE_URL = "https://fbref.com/"
PREM_URL = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [41]:
player_name = "douglas luiz"
player_team = "aston villa"

In [42]:
def get_teams_urls(league_url, table_tag_id = "results2022-202391_overall"):
    # make a request to the webpage. SHOULD MOVE THIS OUT
    response = requests.get(league_url)

    # Set up the Selenium web driver (replace with the path to your chromedriver)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

    # Load the page using Selenium
    driver.get(league_url)

    # create a Beautiful Soup object from the response content
    soup = BeautifulSoup(response.content, 'html.parser')
    
    wait = WebDriverWait(driver, 10)
    table = wait.until(EC.presence_of_element_located((By.ID, table_tag_id)))

    # find the table with id 'div_matchlogs_for'
    table = soup.find('table', {'id': table_tag_id})

    # extract the table rows
    rows = table.find_all('tr')

    # extract the data from each row and store it in a list
    url_data = {}
    for row in rows:
        team_cell = row.find('td', {'data-stat': 'team'})
        if team_cell:
            team_name = team_cell.find("a").text
            team_url_tail = team_cell.find("a")['href']
            team_url = urljoin(FBREF_HOMEPAGE_URL, team_url_tail)
            url_data[team_name] = team_url
            
    driver.quit()
    return url_data


In [44]:
get_teams_urls(PREM_URL)

{'Arsenal': 'https://fbref.com/en/squads/18bb7c10/Arsenal-Stats',
 'Manchester City': 'https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats',
 'Manchester Utd': 'https://fbref.com/en/squads/19538871/Manchester-United-Stats',
 'Tottenham': 'https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats',
 'Liverpool': 'https://fbref.com/en/squads/822bd0ba/Liverpool-Stats',
 'Newcastle Utd': 'https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats',
 'Fulham': 'https://fbref.com/en/squads/fd962109/Fulham-Stats',
 'Brighton': 'https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats',
 'Brentford': 'https://fbref.com/en/squads/cd051869/Brentford-Stats',
 'Chelsea': 'https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats',
 'Aston Villa': 'https://fbref.com/en/squads/8602292d/Aston-Villa-Stats',
 'Crystal Palace': 'https://fbref.com/en/squads/47c64c55/Crystal-Palace-Stats',
 'Wolves': 'https://fbref.com/en/squads/8cec06e1/Wolverhampton-Wanderers-Stats',
 "Nott'ham Forest": '